In [1]:
import re, sys, os
import logging
from pathlib import Path
from datetime import datetime
import numpy as np
from plum import dispatch
import torch
import torch_directml
from transformers import *
from fastai.text.all import *
from fastai.data.all import *
import pandas as pd

torch._logging.set_logs(all=logging.WARNING)

🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py


In [2]:
# Setup DirectML
print("="*80)
print("DIRECTML FULL DEBUG LOG STARTED")
print(f"torch version: {torch_directml.torch.__version__}")
print(f"Device: {torch_directml.device()}")
print(f"GPU: {torch_directml.device_name(0)}")
print("="*80)

dml = torch_directml.device()

# Global monkey-patch for Normalize
old_init = Normalize.__init__
def new_init(self, mean, std, axes=(0,2,3)):
    old_init(self, mean.to(dml), std.to(dml), axes=axes)
Normalize.__init__ = new_init

DIRECTML FULL DEBUG LOG STARTED
torch version: 2.4.1+cpu
Device: privateuseone:0
GPU: Radeon RX 560X 


In [3]:
from fastai.learner import Learner

def directml_safe_freeze_to(self: Learner, n: int):
    """DirectML-safe freeze that works with HFModelWrapper"""
    # For HFModelWrapper: model[0] = backbone, model[1] = classifier
    if isinstance(self.model, HFModelWrapper):
        if n == -1:  # Freeze everything except head
            # Freeze the transformer backbone
            for p in self.model.hf_model.base_model.parameters():
                p.requires_grad_(False)
            # Unfreeze the classifier head
            if hasattr(self.model.hf_model, 'classifier'):
                for p in self.model.hf_model.classifier.parameters():
                    p.requires_grad_(True)
        else:  # Unfreeze everything
            for p in self.model.parameters():
                p.requires_grad_(True)
    else:
        # Standard FastAI behavior for other models
        if n == 0:
            for p in self.model.parameters():
                p.requires_grad_(True)
        else:
            backbone = self.model[0] if hasattr(self.model, '__getitem__') else self.model
            backbone_ids = {id(p) for p in backbone.parameters()}
            for p in backbone.parameters():
                p.requires_grad_(False)
            for p in self.model.parameters():
                if id(p) not in backbone_ids:
                    p.requires_grad_(True)
    
    # Don't recreate optimizer - DirectML issue
    if self.opt is not None:
        self.opt.clear_state()
    
    print(f"✓ Freeze applied (n={n})")

# Patch the learner
Learner.freeze_to = directml_safe_freeze_to

In [4]:
# =====================================================
# 1. Monkey-patch TensorText to add truncate and show methods
# =====================================================
@patch
def truncate(self: TensorText, n):
    """Truncate TensorText to n tokens"""
    return type(self)(self[:n])

@patch  
def show(self: TensorText, ctx=None, **kwargs):
    """Show decoded text"""
    # Get the tokenizer from kwargs or use a stored one
    tokenizer = kwargs.get('tokenizer', getattr(self, '_tokenizer', None))
    
    if tokenizer is not None:
        # Decode using HF tokenizer
        text = tokenizer.decode(self.cpu().tolist(), skip_special_tokens=True)
    else:
        # Fallback: just show token IDs
        text = str(self.cpu().tolist())
    
    if ctx is None:
        print(text)
        return text
    return show_title(text, ctx=ctx, **kwargs)

# =====================================================
# 2. Simple Transform - just tokenize to input_ids
# =====================================================
class HFTokenize(Transform):
    """Transform that tokenizes text files using HuggingFace tokenizer"""
    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        # Store tokenizer globally for TensorText.show() to access
        TensorText._hf_tokenizer = tokenizer
    
    def encodes(self, path: Path):
        """Read file and tokenize, returning TensorText with just input_ids"""
        text = path.read_text(encoding='utf-8') if path.exists() else ""
        
        # Tokenize - get tensors directly from tokenizer
        encoding = self.tokenizer(
            text,
            truncation='longest_first',
            max_length=self.max_length,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='pt'  # Return PyTorch tensors
        )
        
        # Squeeze batch dimension [1, seq_len] -> [seq_len]
        tensor_text = TensorText(encoding['input_ids'].squeeze(0))
        # Attach tokenizer for later decoding
        tensor_text._tokenizer = self.tokenizer
        return tensor_text
    
    def decodes(self, o):
        """Keep as TensorText for show_batch compatibility"""
        # Don't decode to string - keep as TensorText
        # The actual text decoding happens in show() method
        return o

# =====================================================
# 2. Callback that computes attention_mask on-the-fly
# =====================================================
class HFCallback(Callback):
    """
    Computes attention_mask from padded input_ids and injects into model.
    This is the ONLY place where HuggingFace-specific logic lives.
    """
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
    
    def before_batch(self):
        """Compute attention_mask from input_ids before each forward pass"""
        # Get input_ids from batch
        xb = self.xb[0] if isinstance(self.xb, tuple) and len(self.xb) > 0 else self.xb
        
        # Compute attention_mask: 1 for real tokens, 0 for padding
        attention_mask = (xb != self.pad_idx).long()
        
        # Inject into model for this forward pass
        self.model._attention_mask = attention_mask

# =====================================================
# 3. Model wrapper that uses injected attention_mask
# =====================================================
class HFModelWrapper(Module):
    """Wrapper to adapt HF model to FastAI"""
    def __init__(self, hf_model, pad_idx):
        self.hf_model = hf_model
        self.pad_idx = pad_idx
        self._attention_mask = None
    
    def forward(self, input_ids):
        # Get attention_mask from callback (or compute default)
        attention_mask = getattr(self, '_attention_mask', None)
        if attention_mask is None:
            attention_mask = (input_ids != self.pad_idx).long()
        
        # Clear after use (for next batch)
        self._attention_mask = None
        
        # Forward pass with attention mask
        outputs = self.hf_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        return outputs.logits

    def __getitem__(self, idx):
        """Make model subscriptable for FastAI's freeze_to"""
        if idx == 0:
            # Return the transformer backbone (everything except classifier)
            return self.hf_model.base_model
        else:
            # Return the classifier head
            return self.hf_model.classifier

In [5]:
# =====================================================
# 5. Setup DataLoaders and Model
# =====================================================
os.makedirs("models", exist_ok=True)
model_cache_dir = Path("models")
hf_model = "haisongzhang/roberta-tiny-cased"

# Load HF model and tokenizer
hf_model_obj = AutoModelForSequenceClassification.from_pretrained(
    hf_model, num_labels=2, cache_dir=model_cache_dir,
    use_safetensors=True
)
tokenizer = AutoTokenizer.from_pretrained(hf_model)
""

loading configuration file config.json from cache at models\models--haisongzhang--roberta-tiny-cased\snapshots\2e8caf4404987b8cb20fa4b22955f56940b2ebc6\config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

Attempting to create safetensors variant
Safetensors PR exists
loading weights file model.safetensors from cache at models\models--haisongzhang--roberta-tiny-cased\snapshots\bfd314bd7663eef65c83e57719c0d9f3c21aa631\model.sa

''

In [6]:
# Load data
path = untar_data(URLs.IMDB)

# Create DataBlock with HFTransform
blocks = (
    TransformBlock(
        type_tfms=HFTokenize(tokenizer, max_length=512),
        dls_kwargs={'before_batch': Pad_Chunk(seq_len=512, pad_idx=tokenizer.pad_token_id)}
    ),
    CategoryBlock
)

dblock = DataBlock(
    blocks=blocks,
    get_items=get_text_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(valid_name='test')
)

In [7]:
# Create dataloaders
dls = dblock.dataloaders(path, bs=1, device=dml, verbose=True, num_workers=0)

Found 100002 items
2 datasets of sizes 25000,25000
Setting up Pipeline: HFTokenize
Setting up Pipeline: parent_label -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: Pad_Chunk -- {'pad_idx': 0, 'pad_first': True, 'seq_len': 512}

Setting up after_batch: Pipeline: 


In [8]:
dls.show_batch(max_n=2)

,None,category
0,"well when watching this film late one night i was simple amazed by it's greatness. fantastic script, great acting, costumes and special effects, and the plot twists, wow!! in fact if you can see the ending coming you should become a writer yourself. < br / > < br / > great, i would recommend this film to anyone, especially if i don ; t like them much. < br / > < br / > terrific",pos


In [9]:
# =====================================================
# 7. Create Learner with custom callback
# =====================================================
# Wrap the HF model
model_wrapped = HFModelWrapper(hf_model_obj, tokenizer.pad_token_id)
model_wrapped.to(dml)

# Create learner
learn = Learner(
    dls, 
    model_wrapped,
    loss_func=CrossEntropyLossFlat(),
    metrics=accuracy,
    cbs=[HFCallback(pad_idx=tokenizer.pad_token_id)]
).to_fp16(enabled=False)

learn.to(dml)
print("\n" + "="*80)
print("LEARNER CREATED SUCCESSFULLY")
print("="*80)


LEARNER CREATED SUCCESSFULLY


In [10]:
print("\n" + "="*80)
print("VERIFYING BATCH STRUCTURE")
print("="*80)

try:
    batch = dls.one_batch()
    xb, yb = batch
    
    print(f"Batch shape: {xb.shape}")
    print(f"Batch dtype: {xb.dtype}")
    print(f"First item type: {type(xb[0])}")
    print(f"Labels shape: {yb.shape}")
    print(f"Pad token ID: {tokenizer.pad_token_id}")
    print(f"Sample tokens (first 20): {xb[0][:20]}")
    
    # Show padding
    pad_count = (xb[0] == tokenizer.pad_token_id).sum().item()
    print(f"Padding tokens in first sample: {pad_count}/{len(xb[0])}")
    
    print("\n✓ Batch structure correct!")
except Exception as e:
    print(f"\n✗ Batch verification error: {e}")
    import traceback
    traceback.print_exc()


VERIFYING BATCH STRUCTURE
Batch shape: torch.Size([1, 512])
Batch dtype: torch.int64
First item type: <class 'fastai.text.data.TensorText'>
Labels shape: torch.Size([1])
Pad token ID: 0
Sample tokens (first 20): TensorText([  101,  1175,  1110,  7284,  1720,  1106,  1894,  3051,  1306,
             1142,  2523,   119,  1152,  1261,   170,   188, 19094,  6482,
             1642,   117], device='privateuseone:0')
Padding tokens in first sample: 276/512

✓ Batch structure correct!


In [11]:
# # =====================================================
# # 9. Test forward pass with attention_mask injection
# # =====================================================
# print("\n" + "="*80)
# print("TESTING FORWARD PASS WITH ATTENTION MASK")
# print("="*80)

# try:
#     with torch.no_grad():
#         batch = dls.one_batch()
#         learn.xb, learn.yb = batch
        
#         # Find our HFCallback
#         hf_callback = None
#         for cb in learn.cbs:
#             if isinstance(cb, HFCallback):
#                 hf_callback = cb
#                 break
        
#         print(f"Found HFCallback: {hf_callback is not None}")
        
#         # Show what callback does
#         print("\nBefore callback:")
#         print(f"  Model has _attention_mask: {hasattr(learn.model, '_attention_mask')}")
#         print(f"  Input batch shape: {learn.xb[0].shape}")
#         print(f"  Input batch type: {type(learn.xb[0])}")
        
#         # Trigger callback manually
#         if hf_callback:
#             hf_callback.before_batch()
            
#             print("\nAfter callback:")
#             print(f"  Model has _attention_mask: {hasattr(learn.model, '_attention_mask')}")
#             if hasattr(learn.model, '_attention_mask'):
#                 print(f"  Attention mask shape: {learn.model._attention_mask.shape}")
#                 print(f"  Sample attention mask (first 30): {learn.model._attention_mask[0][:30]}")
#                 print(f"  Num real tokens: {learn.model._attention_mask[0].sum().item()}")
#                 print(f"  Num pad tokens: {(learn.model._attention_mask[0] == 0).sum().item()}")
        
#         # Forward pass - pass the whole batch, not just first element
#         input_tensor = learn.xb[0] if isinstance(learn.xb, tuple) else learn.xb
#         pred = learn.model(input_tensor)
        
#         print("\nForward pass results:")
#         print(f"  Input shape: {input_tensor.shape}")
#         print(f"  Output shape: {pred.shape}")
#         print(f"  Sample logits: {pred[0]}")
#         print("\n✓ Forward pass successful!")
        
# except Exception as e:
#     print(f"\n✗ Forward pass error: {e}")
#     import traceback
#     traceback.print_exc()

In [12]:
learn.fine_tune(1, base_lr=3e-4)

✓ Freeze applied (n=-1)


f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,time
0,0.439975,0.441304,0.795480,1:43:13


✓ Freeze applied (n=0)


epoch,train_loss,valid_loss,accuracy,time
0,0.479586,0.422859,0.804920,2:48:38


In [14]:
test_path = Path("test_reviews/test_1.txt")
learn.predict(test_path)

f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


('pos', TensorText(1), TensorText([0.1503, 0.8497]))